In [ ]:
audio_files_directory="" #GCS URI to audio files in format gs://bucket_name/path, files must be stored in subdirectories named after language code of the video
gcs_transcripts_directory="" #GCS URI to store generated transcripts in format gs://bucket_name/path
model="medium" #Whisper model to use for transcription (i.e. tiny, base, small, medium, long)

In [ ]:
!(sudo apt update && sudo apt install ffmpeg)
!(pip install -U openai-whisper)

In [ ]:
import re

language_directories=!(gsutil ls {audio_files_directory})
file_array=[]

for language_directory in language_directories:
    
    language_code=re.search("/([^/]+)/$",language_directory).group(1)
    files=!(gsutil ls {language_directory})
    for uri in files:
        
        file_array.append({"gcs_uri": uri,"language_code": language_code})

print(file_array)
number_of_files=len(file_array)
print(f"Number of files: {number_of_files}")

In [ ]:
import re
from google.cloud import storage
import os

storage_client=storage.Client()
bucket_name=re.search(f"gs://([^/]+)/",audio_files_directory).group(1)
bucket = storage_client.bucket(bucket_name)

for file in file_array:
    regex=f"gs://{bucket_name}/(.*)/(\w+.flac)$"
    path,filename=re.search(f"gs://{bucket_name}/(.*)/(\w+.flac)$",file["gcs_uri"]).group(1,2)
    local_uri=f"sound/{file['language_code']}/{filename}"
    # Fetch results from Cloud Storage
    blob = bucket.blob(f"{path}/{filename}")
    
    if not os.path.exists(f"sound/{file['language_code']}/"):
            os.makedirs(f"sound/{file['language_code']}/")
    blob.download_to_filename(local_uri)
    file["local_uri"]=local_uri
    
print(file_array)

In [ ]:
import whisper
import time
import re
from google.cloud import storage

whisper_model=whisper.load_model(model)

storage_client=storage.Client()
bucket_name, transcript_path=re.search(f"gs://([^/]+)/(.*)",gcs_transcripts_directory).group(1, 2)
bucket = storage_client.bucket(bucket_name)

start_time = time.time()

for file in file_array:
    transcript=whisper_model.transcribe(file["local_uri"], fp16=False)
    print(f"Transcript for {file['gcs_uri']}: {transcript['text']}")

    transcript_id=re.search("([^/]+)\.flac$",file["gcs_uri"]).group(1)
    
    blob=bucket.blob(f"{transcript_path}/whisper_{model}/txt/{transcript_id}.txt")
    blob.upload_from_string(transcript["text"])
    
duration = time.time() - start_time
print(f"Took {duration} to execute")